In [0]:
import numpy as np
import pandas as pd 
import requests 
from bs4 import BeautifulSoup 


In [136]:
#Web scraping
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(URL) 
print(page)   
soup = BeautifulSoup(page.content, 'html.parser') 


<Response [200]>


Read the table


In [0]:
#get the value from the table
postalCodes = [];
boroughs= [];
neighborhoods = [];
for row in soup.find('table',class_="wikitable").find_all('tr'):
   
    x=row.find_all('td')
    if len(x)!=0 :
      if x[1].text!='Not assigned\n':
        
        postalCodes.append(x[0].text)
        boroughs.append(x[1].text)
        neighborhoods.append(x[2].text)


In [0]:
# define the dataframe 
neighbors = pd.DataFrame({'PostalCode':postalCodes, 'Borough':boroughs, 'Neighborhood':neighborhoods})
#clean data
neighbors = neighbors.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)
neighbors =neighbors.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r' / ',  ',', regex=True)

In [139]:
neighbors.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park,Ontario Provincial Government"


In [140]:
neighbors.shape

(103, 3)

## **Get** **Cordinates**

In [141]:
import sys
!{sys.executable} -m pip install geocoder

In [0]:
import geocoder 

import requests

In [0]:
lat_lng_coords = None
latitude=[];
longitude=[];
for code in range(0, len(neighbors['PostalCode'])):
      
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])

In [143]:
neighbors['Latitude']=latitude
neighbors['Longitude']=longitude
neighbors.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.648690,-79.385440
1,M4A,North York,Victoria Village,43.657993,-79.353938
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.638500,-79.422230
3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.683396,-79.362194
4,M7A,Downtown Toronto,"Queen's Park,Ontario Provincial Government",43.643580,-79.395720


In [144]:
neighbors.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.648690,-79.385440
1,M4A,North York,Victoria Village,43.657993,-79.353938
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.638500,-79.422230
3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.683396,-79.362194
4,M7A,Downtown Toronto,"Queen's Park,Ontario Provincial Government",43.643580,-79.395720
5,M9A,Etobicoke,Islington Avenue,44.005423,-77.888860
6,M1B,Scarborough,"Malvern,Rouge",43.776920,-79.345090
7,M3B,North York,Don Mills,43.581744,-81.519861
8,M4B,East York,"Parkview Hill,Woodbine Gardens",43.543637,-80.238396
9,M5B,Downtown Toronto,"Garden District, Ryerson",44.005650,-77.888997


# Dataset exploration

In [87]:
!{sys.executable} -m pip install folium

In [0]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [148]:
'{}, {}'.format(neighbors['Neighborhood'][0], neighbors['Borough'][0])

'Parkwoods, North York'

In [149]:
## Create a map of Toronto using latitude and longitude values 
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)
# add markers to map
for i in range(0,len(neighbors)):
    label = '{}, {}'.format(neighbors['Neighborhood'][i], neighbors['Borough'][i])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [neighbors['Latitude'][i], neighbors['Longitude'][i]],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [150]:
VERSION = '20180605' # Foursquare API version

neighborhood_name = neighbors.loc[0, 'Neighborhood'] # neighborhood name
neighborhood_latitude = neighbors.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighbors.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
        # https://api.foursquare.com/v2/venues/search?ll=40.7,-74&client_id=CLIENT_ID&client_secret=CLIENT_SECRET&v=YYYYMMDD
# url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#     CLIENT_ID, XD0MCDPMOCYAVDK3K20GIK33WTNUG0RNGYVRWFEOQTKDQCVI
#     CLIENT_SECRET, ICMDJNT2ESQQ1UID1CET3ZX43ORMPCQQHP2KA3KQWOHG4Z55
#     VERSION, 
#     43.67635739999999, 
#     79.2930312, 
#     radius, 
#     LIMIT)

url = 'https://api.foursquare.com/v2/venues/explore?ll=43.67635739999999,-79.2930312&client_id=XD0MCDPMOCYAVDK3K20GIK33WTNUG0RNGYVRWFEOQTKDQCVI&client_secret=ICMDJNT2ESQQ1UID1CET3ZX43ORMPCQQHP2KA3KQWOHG4Z55&v=20180605&radius=500&limit=100'

results = requests.get(url).json()
results

Latitude and longitude values of Parkwoods are 43.648690000000045, -79.38543999999996.


{'meta': {'code': 200, 'requestId': '5e8650e9006dce001b82566b'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bd461bc77b29c74a07d9282-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/hikingtrail_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d159941735',
         'name': 'Trail',
         'pluralName': 'Trails',
         'primary': True,
         'shortName': 'Trail'}],
       'id': '4bd461bc77b29c74a07d9282',
       'location': {'address': 'Glen Manor',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Queen St.',
        'distance': 89,
        'formattedAddress': ['Glen Manor (Queen St.)', 'Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682

In [0]:
# From the Foursquare lab in the previous module, we know that all the information is in the items key. 
# Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [152]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Glen Stewart Park,Park,43.675278,-79.294647
4,Upper Beaches,Neighborhood,43.680563,-79.292869


In [153]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


Explore all neighborhoods in Toronto

In [0]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
#         url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#             CLIENT_ID, 
#             CLIENT_SECRET, 
#             VERSION, 
#             lat, 
#             lng, 
#             radius, 
#             LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [155]:
toronto_venues = getNearbyVenues(names=neighborhoods,
                                 latitudes=neighbors['Latitude'],
                                 longitudes=neighbors['Longitude'])

Parkwoods

Victoria Village

Regent Park / Harbourfront

Lawrence Manor / Lawrence Heights

Queen's Park / Ontario Provincial Government

Islington Avenue

Malvern / Rouge

Don Mills

Parkview Hill / Woodbine Gardens

Garden District, Ryerson

Glencairn

West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale

Rouge Hill / Port Union / Highland Creek

Don Mills

Woodbine Heights

St. James Town

Humewood-Cedarvale

Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood

Guildwood / Morningside / West Hill

The Beaches

Berczy Park

Caledonia-Fairbanks

Woburn

Leaside

Central Bay Street

Christie

Cedarbrae

Hillcrest Village

Bathurst Manor / Wilson Heights / Downsview North

Thorncliffe Park

Richmond / Adelaide / King

Dufferin / Dovercourt Village

Scarborough Village

Fairview / Henry Farm / Oriole

Northwood Park / York University

East Toronto

Harbourfront East / Union Station / Toronto Islands

Little Portugal / Trinity

Kennedy Park / Ionview / E

In [156]:
# check the size of the resulting dataframe

print(toronto_venues.shape)
toronto_venues.head()

(515, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods\n,43.64869,-79.38544,Glen Manor Ravine,43.676821,-79.293942,Trail
1,Parkwoods\n,43.64869,-79.38544,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,Parkwoods\n,43.64869,-79.38544,Grover Pub and Grub,43.679181,-79.297215,Pub
3,Parkwoods\n,43.64869,-79.38544,Glen Stewart Park,43.675278,-79.294647,Park
4,Parkwoods\n,43.64869,-79.38544,Upper Beaches,43.680563,-79.292869,Neighborhood


In [157]:
# check how many venues were returned for each neighborhood

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt\n,5,5,5,5,5,5
Alderwood / Long Branch\n,5,5,5,5,5,5
Bathurst Manor / Wilson Heights / Downsview North\n,5,5,5,5,5,5
Bayview Village\n,5,5,5,5,5,5
Bedford Park / Lawrence Manor East\n,5,5,5,5,5,5
...,...,...,...,...,...,...
Willowdale / Newtonbrook\n,5,5,5,5,5,5
Woburn\n,5,5,5,5,5,5
Woodbine Heights\n,5,5,5,5,5,5


In [158]:
# find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 5 uniques categories.


In [159]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Trail,Health Food Store,Neighborhood,Park,Pub
0,1,0,Parkwoods\n,0,0
1,0,1,Parkwoods\n,0,0
2,0,0,Parkwoods\n,0,1
3,0,0,Parkwoods\n,1,0
4,0,0,Parkwoods\n,0,0


In [160]:
# examine the new dataframe size.

toronto_onehot.shape

(515, 5)

In [161]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Trail,Health Food Store,Park,Pub
0,Agincourt\n,0.2,0.2,0.2,0.2
1,Alderwood / Long Branch\n,0.2,0.2,0.2,0.2
2,Bathurst Manor / Wilson Heights / Downsview No...,0.2,0.2,0.2,0.2
3,Bayview Village\n,0.2,0.2,0.2,0.2
4,Bedford Park / Lawrence Manor East\n,0.2,0.2,0.2,0.2
...,...,...,...,...,...
93,Willowdale / Newtonbrook\n,0.2,0.2,0.2,0.2
94,Woburn\n,0.2,0.2,0.2,0.2
95,Woodbine Heights\n,0.2,0.2,0.2,0.2
96,York Mills / Silver Hills\n,0.2,0.2,0.2,0.2


In [162]:
# confirm the new size

toronto_grouped.shape

(98, 5)

In [164]:
# print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt
----
               venue  freq
0              Trail   0.2
1  Health Food Store   0.2
2               Park   0.2
3                Pub   0.2


----Alderwood / Long Branch
----
               venue  freq
0              Trail   0.2
1  Health Food Store   0.2
2               Park   0.2
3                Pub   0.2


----Bathurst Manor / Wilson Heights / Downsview North
----
               venue  freq
0              Trail   0.2
1  Health Food Store   0.2
2               Park   0.2
3                Pub   0.2


----Bayview Village
----
               venue  freq
0              Trail   0.2
1  Health Food Store   0.2
2               Park   0.2
3                Pub   0.2


----Bedford Park / Lawrence Manor East
----
               venue  freq
0              Trail   0.2
1  Health Food Store   0.2
2               Park   0.2
3                Pub   0.2


----Berczy Park
----
               venue  freq
0              Trail   0.2
1  Health Food Store   0.2
2               Park   0.2
3     

In [0]:
# sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [166]:
# create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Agincourt\n,Pub,Park,Health Food Store
1,Alderwood / Long Branch\n,Pub,Park,Health Food Store
2,Bathurst Manor / Wilson Heights / Downsview No...,Pub,Park,Health Food Store
3,Bayview Village\n,Pub,Park,Health Food Store
4,Bedford Park / Lawrence Manor East\n,Pub,Park,Health Food Store


In [172]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [174]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighbors

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M3A,North York,Parkwoods,43.648690,-79.385440,NaN,NaN,NaN,NaN
1,M4A,North York,Victoria Village,43.657993,-79.353938,NaN,NaN,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.638500,-79.422230,NaN,NaN,NaN,NaN
3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.683396,-79.362194,NaN,NaN,NaN,NaN
4,M7A,Downtown Toronto,"Queen's Park,Ontario Provincial Government",43.643580,-79.395720,NaN,NaN,NaN,NaN


In [0]:
# Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[43.67635739999999, -79.2930312], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [170]:
# CLUSTER 1

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
